In [6]:
import cv2
import mediapipe as mp
import time
import numpy as np
from PIL import Image, ImageSequence

# --- PENGATURAN ---
CHARACTER_IMAGE_PATH = './assets/denis-gerak-noBG.gif' # Path ke file GIF (Jalan Kiri)
IDLE_IMAGE_PATH = './assets/denis_diam.png'        # Path ke gambar diam (Hadap Kiri)
CHARACTER_SIZE = (128, 128)
MOVE_SPEED = 7
DEAD_ZONE_SIZE = 35
GAME_BG_COLOR = (0, 0, 0)
ANIMATION_DELAY = 0.1
# --------------------

# Fungsi overlay_image_alpha (Sama)
def overlay_image_alpha(background, overlay, x, y, size):
    if overlay is None: return
    try:
        overlay_resized = cv2.resize(overlay, size)
    except cv2.error as e:
        print(f"Error resizing overlay: {e}"); return

    img_h, img_w, img_c = overlay_resized.shape
    bg_h, bg_w, _ = background.shape
    x_tl = x - img_w // 2; y_tl = y - img_h // 2
    x1, x2 = max(0, x_tl), min(bg_w, x_tl + img_w)
    y1, y2 = max(0, y_tl), min(bg_h, y_tl + img_h)
    ix1, ix2 = max(0, -x_tl), min(img_w, bg_w - x_tl)
    iy1, iy2 = max(0, -y_tl), min(img_h, bg_h - y_tl)
    if x1 >= x2 or y1 >= y2 or ix1 >= ix2 or iy1 >= iy2: return

    roi = background[y1:y2, x1:x2]
    overlay_crop = overlay_resized[iy1:iy2, ix1:ix2]
    if roi.shape[0] != overlay_crop.shape[0] or roi.shape[1] != overlay_crop.shape[1]:
        overlay_crop = cv2.resize(overlay_crop, (roi.shape[1], roi.shape[0]))

    if img_c == 4:
        alpha_s = overlay_crop[:, :, 3] / 255.0; alpha_l = 1.0 - alpha_s
        for c in range(0, 3):
            roi[:, :, c] = (alpha_s * overlay_crop[:, :, c] + alpha_l * roi[:, :, c])
    else:
        background[y1:y2, x1:x2] = overlay_crop[:,:,:3]

# Fungsi load_gif_frames (Sama)
def load_gif_frames(gif_path):
    frames_cv2 = []
    try:
        with Image.open(gif_path) as img:
            print(f"Memuat GIF: {gif_path}...")
            for frame_pil in ImageSequence.Iterator(img):
                frame_pil = frame_pil.convert('RGBA')
                frame_np = np.array(frame_pil)
                frame_bgra = cv2.cvtColor(frame_np, cv2.COLOR_RGBA2BGRA)
                frames_cv2.append(frame_bgra)
            print(f"Berhasil memuat {len(frames_cv2)} frame GIF.")
    except Exception as e:
        print(f"Error saat memuat GIF {gif_path}: {e}")
    return frames_cv2

# Fungsi load_idle_image (Sama)
def load_idle_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None: print(f"Error: Gagal memuat {image_path}"); return None
    h, w, *c = img.shape
    if not c: img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGRA)
    elif c[0] == 3: img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    print(f"Berhasil memuat gambar diam: {image_path}")
    return img

# --- Muat Frame Karakter & Gambar Diam ---
character_frames = load_gif_frames(CHARACTER_IMAGE_PATH)
idle_frame = load_idle_image(IDLE_IMAGE_PATH)

# Fallback
fallback_img = np.zeros((100, 100, 4), dtype=np.uint8)
cv2.circle(fallback_img, (50, 50), 40, (0, 255, 0, 255), -1)
if not character_frames: character_frames.append(fallback_img)
if idle_frame is None: idle_frame = character_frames[0]

# --- <<< BARU: Buat Frame yang Dibalik (Flip) >>> ---
character_frames_flipped = [cv2.flip(frame, 1) for frame in character_frames]
idle_frame_flipped = cv2.flip(idle_frame, 1)
# ---------------------------------------------------

# Inisialisasi MediaPipe & OpenCV (Sama)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
success, temp_frame = cap.read()
if not success: print("Tidak bisa mengakses webcam."); exit()
height, width, _ = temp_frame.shape
temp_frame = None

# Posisi awal objek (Sama)
obj_x, obj_y = width // 2, height // 2

# === Variabel Pusat Referensi (Sama) ===
initial_nose_x = None
initial_nose_y = None
center_set = False
# ============================================

# --- Variabel Animasi & Arah ---
current_frame_index = 0
last_frame_time = time.time()
num_frames = len(character_frames)
last_direction = "KIRI" # <<< BARU: Menyimpan arah terakhir (KIRI atau KANAN)
# -----------------------------------

print("Arahkan wajah ke tengah dan tunggu hingga garis silang muncul...")

while cap.isOpened():
    success, image = cap.read()
    if not success: print("Gagal membaca frame."); continue

    game_window = np.full((height, width, 3), GAME_BG_COLOR, dtype=np.uint8)
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    move_x = 0
    move_y = 0
    status_text = "CARI WAJAH..."

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            nose_tip = face_landmarks.landmark[1]
            current_nose_x = int(nose_tip.x * width)
            current_nose_y = int(nose_tip.y * height)

            if not center_set:
                initial_nose_x = current_nose_x
                initial_nose_y = current_nose_y
                center_set = True
                print(f"Pusat Referensi Diatur: ({initial_nose_x}, {initial_nose_y})")

            cv2.circle(image, (current_nose_x, current_nose_y), 5, (0, 0, 255), -1)

            if center_set:
                dead_zone_x1 = initial_nose_x - DEAD_ZONE_SIZE // 2
                dead_zone_x2 = initial_nose_x + DEAD_ZONE_SIZE // 2
                dead_zone_y1 = initial_nose_y - DEAD_ZONE_SIZE // 2
                dead_zone_y2 = initial_nose_y + DEAD_ZONE_SIZE // 2

                # --- Logika Gerakan (Dengan Update last_direction) ---
                if current_nose_y > dead_zone_y2:
                    move_y = MOVE_SPEED; move_x = 0; status_text = "BAWAH"
                    # Arah horizontal tidak berubah saat gerak vertikal
                elif current_nose_y < dead_zone_y1:
                    move_y = -MOVE_SPEED; move_x = 0; status_text = "ATAS"
                    # Arah horizontal tidak berubah saat gerak vertikal
                else:
                    if current_nose_x > dead_zone_x2:
                        move_x = MOVE_SPEED; move_y = 0; status_text = "KANAN"
                        last_direction = "KANAN" # <<< Update arah
                    elif current_nose_x < dead_zone_x1:
                        move_x = -MOVE_SPEED; move_y = 0; status_text = "KIRI"
                        last_direction = "KIRI" # <<< Update arah
                    else:
                        move_x = 0; move_y = 0; status_text = "BERHENTI"
                # ----------------------------------------------------

                cv2.line(image, (initial_nose_x, 0), (initial_nose_x, height), (0, 255, 255), 2)
                cv2.line(image, (0, initial_nose_y), (width, initial_nose_y), (0, 255, 255), 2)
                cv2.rectangle(image, (dead_zone_x1, dead_zone_y1), (dead_zone_x2, dead_zone_y2), (255, 0, 0), 1)
            else:
                 status_text = "TUNGGU PUSAT DIATUR..."
    else:
        move_x = 0; move_y = 0; status_text = "CARI WAJAH..."

    # Logika Animasi (Sama)
    current_time = time.time()
    if status_text != "BERHENTI" and num_frames > 1 and (current_time - last_frame_time > ANIMATION_DELAY):
        current_frame_index = (current_frame_index + 1) % num_frames
        last_frame_time = current_time

    obj_x += move_x
    obj_y += move_y
    char_w, char_h = CHARACTER_SIZE
    obj_x = max(char_w // 2, min(width - char_w // 2, obj_x))
    obj_y = max(char_h // 2, min(height - char_h // 2, obj_y))

    # --- <<< Pilih Frame Berdasarkan Arah (DIUBAH) >>> ---
    if status_text == "BERHENTI":
        frame_to_draw = idle_frame_flipped if last_direction == "KANAN" else idle_frame
    elif status_text == "KANAN":
        frame_to_draw = character_frames_flipped[current_frame_index]
    elif status_text == "KIRI":
        frame_to_draw = character_frames[current_frame_index]
    else: # ATAS atau BAWAH - Gunakan arah terakhir
        frame_to_draw = character_frames_flipped[current_frame_index] if last_direction == "KANAN" else character_frames[current_frame_index]
    # ----------------------------------------------------

    # Gambar Karakter
    overlay_image_alpha(game_window, frame_to_draw, obj_x, obj_y, CHARACTER_SIZE)

    cv2.putText(image, f"Status: {status_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Controller', image)
    cv2.imshow('Game', game_window)

    if cv2.waitKey(5) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

Memuat GIF: ./assets/denis-gerak-noBG.gif...
Berhasil memuat 3 frame GIF.
Berhasil memuat gambar diam: ./assets/denis_diam.png
Arahkan wajah ke tengah dan tunggu hingga garis silang muncul...
Pusat Referensi Diatur: (337, 263)
